In [1]:
# How to call the correct_lean function
import json
from lean_correction import correct_lean
from constants import HUMAN_ANNOTATION_PATH

with open(HUMAN_ANNOTATION_PATH) as f:
        _human_dataset = json.load(f)
        correct_lean(_human_dataset)

In [19]:
# How to reprocess the data instead of loading from a save
from postprocess_original_utils import load_data

data = load_data(load_from_file=False)

Read in 601 HIT total assignments
568 good, 33 flagged, 0 error, 0 rejected
Relative depth ordering for Turkers associated with the data
105812 images in human3.6 dataset
568 annotations matched with ground truth
Output data to /Users/Robert/Documents/Caltech/CS81_Depth_Research/results/human36m_processed_data.json


In [ ]:
# Download caltech mini experiment from server. To be run on the server

import json
import sys
from pymongo import MongoClient
_mongo_client = MongoClient()

_mongo_db = _mongo_client.cocoa_depth_human36m

_mongo_coll_1 = _mongo_db.depth_amt_gui_data
_mongo_coll_2 = _mongo_db.keypoint_labels
_mongo_coll_3 = _mongo_db.depth_hit_id2human_subj_id
_mongo_coll_4 = _mongo_db.human_subj_id2depth_hit_id

_mongo_coll_5 = _mongo_db.depth_amt_gui_workers
_mongo_coll_6 = _mongo_db.depth_amt_gui_blocked_workers

_mongo_coll_7 = _mongo_db.depth_amt_gui_trials_results
_mongo_coll_8 = _mongo_db.depth_amt_gui_hits_results

caltech_comments = ["caltech", "oisin", "milan", "jalani", "ron", "amanda", "lucy", "matteo", "jennifer"]
hits = []

cursor = _mongo_coll_8.find({})
for document in cursor:
    # print document
    if document['_hit_comment'] != "":
        # print document['_hit_id']
        if any(comment.lower() in document['_hit_comment'].lower().strip() for comment in caltech_comments):
            # print document['_hit_comment']
            print document['_hit_it']
            print document['_hit_id']
            subj_id = _mongo_coll_3.find_one({'_amt_hit_id': document['_hit_id']})['_human_subjs_ids'][0]
            res_coll_1 = _mongo_coll_1.find_one({ '_human_subj_id': subj_id })
            document['img_id'] = res_coll_1['_human_img_id']
            document['_id'] = str(document['_id'])
            hits.append(document)

f = open("caltech_results.json", "w")
f.write(json.dumps(hits))
f.close()


In [45]:
# Load results from caltech_results into python
import json

OLD_KEYCMPS = "/Users/Robert/Documents/Caltech/EE148/ee148project/keycmps.json"
CALTECH_RESULTS = "/Users/Robert/Documents/Caltech/CS81_Depth_Research/results/caltech_results.json"

caltech_results = json.load(open(CALTECH_RESULTS))
old_keycmps = json.load(open(OLD_KEYCMPS))

In [51]:
# Convert caltech results into something consumable by the local server to get the mergesort ordering
import json
KEYCMPS_RESULT_PATH = "/Users/Robert/Documents/Caltech/CS81_Depth_Research/results/caltech_results_keycmps.json"

temp = [map(json.loads, c['_hit_depth_str'].split("***")[1:]) for c in caltech_results]
keycmps = [item for t in temp for item in t]

keycmp_orders = [k['keypoint_comparisons_order'] for k in keycmps]
keycmp_reses = [k['keypoint_comparisons_res'] for k in keycmps]
keycmps = {'keycmp_orders': keycmp_orders, 'keycmp_reses': keycmp_reses}

with open(KEYCMPS_RESULT_PATH, 'w') as f:
    f.write(json.dumps(keycmps))

In [46]:
import json
# All the comparisons and hits for one person are squished into one hit. Separate them into multiple hits.
unfolded_caltech_results = []
temp = [map(json.loads, c['_hit_depth_str'].split("***")[1:]) for c in caltech_results]
for t, c in zip(temp, caltech_results):
    for trial_ind, cmps in enumerate(t):
        res_copy = c.copy()
        keycmp_order = cmps['keypoint_comparisons_order']
        keycmp_res = cmps['keypoint_comparisons_res']
        # dumb format carried over from before
        res_copy['trials'] = [{'depth':{}}]
        res_copy['trials'][0]['depth']['keypoint_comparisons_order'] = keycmp_order
        res_copy['trials'][0]['depth']['keypoint_comparisons_res'] = keycmp_res
        res_copy['trials'][0]['trial_ind'] = trial_ind
        unfolded_caltech_results.append(res_copy)

# Take the mergesort ordering from the local server and reincorporate it into the original data
KEYPTS_RELATIVE_DEPTH_PATH = "/Users/Robert/Documents/Caltech/CS81_Depth_Research/results/caltech_results_keypts_relative_depth.json"

_relative_depth = json.load(open(KEYPTS_RELATIVE_DEPTH_PATH, 'r'))
for d, kpts_relative_depth in zip(unfolded_caltech_results, _relative_depth):
    d['trials'][0]['kpts_relative_depth'] = kpts_relative_depth

In [47]:
# For the newly separated hits, the img_id must be reassociated with it
# All trial_inds above 0 should line up with this list
miniexp_img_ids = [56833, 965922, 849671, 649263, 12750, 896082, 571176, 965922]

# TODO: FIX THIS BUG. MISALIGNMENT OF NON MINIEXPERIMENT TECHERS
for u in unfolded_caltech_results:
    #if u['trials'][0]['trial_ind'] > 0:
    u['trials'][0]['img_id'] = miniexp_img_ids[u['trials'][0]['trial_ind']]

In [48]:
# Time to line up the hits with the ground truth human data. This is more or less the same as the 
# code from the original load data postprocess.
import json
import copy
from postprocess_original_utils import correct_lean
from constants import HUMAN_ANNOTATION_PATH

img_ids = [c['trials'][0]['img_id'] for c in unfolded_caltech_results]

# Get the ground truth annotations from human dataset
with open(HUMAN_ANNOTATION_PATH) as f:
    _human_dataset = json.load(f)
    correct_lean(_human_dataset)

for i in range(len(_human_dataset['images'])):
    if _human_dataset['images'][i]['id'] in img_ids:
        img_id_idxes = [img_id_idx for img_id_idx, img_id in enumerate(img_ids)
                        if img_id == _human_dataset['images'][i]['id']]
        for img_id_idx in img_id_idxes:
            unfolded_caltech_results[img_id_idx]['images_truth'] = _human_dataset['images'][i]
            unfolded_caltech_results[img_id_idx]['annotations_truth'] = copy.deepcopy(_human_dataset['annotations'][i])

print "{} images in human3.6 dataset".format(len(_human_dataset['images']))
print "{} annotations matched with ground truth".format(len(unfolded_caltech_results))

# Remove the neck keypoint. Arrange the depth data in a useful way.
for d in unfolded_caltech_results:
    # Remove neck keypoint
    d['annotations_truth']['kpts_2d'][2:4] = []
    d['annotations_truth']['kpts_3d'][3:6] = []
    # Grab every third kpts_3d which corresponds to depth
    axis_to_sort = 1 # change this if it looks like we're sorting along the wrong axis
    d['annotations_truth']['kpts_depth'] = d['annotations_truth']['kpts_3d'][axis_to_sort::3]
    d['annotations_truth']['kpts_relative_depth'] = \
        [i[0] for i in sorted(enumerate(d['annotations_truth']['kpts_depth']), key=lambda x:x[1])]


105812 images in human3.6 dataset
78 annotations matched with ground truth


In [49]:
# Small aesthetic rename
for d in unfolded_caltech_results:
    d['worker_id'] = d['_worker_id']

In [50]:
# Finally write it to a file!
import json
CALTECH_OUTPUT_PATH = "/Users/Robert/Documents/Caltech/CS81_Depth_Research/results/human36m_processed_caltech_data.json"

with open(CALTECH_OUTPUT_PATH, 'w') as f:
    json.dump(unfolded_caltech_results, f)
print "Output data to {}".format(CALTECH_OUTPUT_PATH)

Output data to /Users/Robert/Documents/Caltech/CS81_Depth_Research/results/human36m_processed_caltech_data.json


In [24]:
import json
from constants import CALTECH_OUTPUT_PATH
data = json.load(open(CALTECH_OUTPUT_PATH, 'r'))

data[3]['annotations_truth']['i_id']
set([d['_hit_comment'][:20] + ": " + d['_worker_id'] for d in data])
[d for d in data if d['_worker_id'] == 'nonAMT_368102']
data[0].keys()

from constants import HUMAN_OUTPUT_PATH
data = json.load(open(HUMAN_OUTPUT_PATH, 'r'))
data[0].keys()


[u'worker_exp',
 u'hit_id',
 u'assignment_id',
 u'trials',
 u'hit_comment',
 u'annotations_truth',
 u'human_subj_ids',
 u'worker_id',
 u'hit_it',
 u'images_truth',
 u'gui_rating',
 u'response_time']